# End‑to‑End Demo: NumPy → MySQL → SQL Transforms → pandas → CSV

This notebook shows how to:
1. **Generate** a 200×15 synthetic dataset with **NumPy** (wrapped in a **pandas** DataFrame).
2. **Ingest** that data into **MySQL** using `mysql.connector`.
3. **Transform** the data with **3 SQL queries** (aggregations & KPIs).
4. **Load** SQL results back to pandas **DataFrames** and **export** them to `.csv`.

> ⚙️ **Prerequisites**  
> - MySQL server accessible (local or remote).  
> - Python packages: `numpy`, `pandas`, `mysql-connector-python`.  
> - A MySQL user with privileges to create a database/table (or pre-create them).

---


## 1) Prerequisites & Connection Settings

We configure connection details for MySQL and import required libraries.  
Update the placeholders (`host`, `user`, `password`, etc.) to match your environment.

**Tip:** For local development on Linux/macOS with Homebrew or apt-installed MySQL, the host is often `localhost` and port `3306`.  
On managed services (e.g., RDS), use the provided host, port, and SSL settings if required.


In [ ]:
# If needed, install packages (uncomment and run once)
# %pip install numpy pandas mysql-connector-python

import os
import numpy as np
import pandas as pd
import mysql.connector as mysql
from datetime import datetime, timedelta

# --- MySQL connection parameters ---
MYSQL_HOST = os.getenv("MYSQL_HOST", "localhost")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", "3306"))
MYSQL_USER = os.getenv("MYSQL_USER", "demo_user")      # <-- change me
MYSQL_PASS = os.getenv("MYSQL_PASS", "demo_password")  # <-- change me
MYSQL_DB   = os.getenv("MYSQL_DB",   "demo_db")        # database to create/use

print(f"Will connect to MySQL at {MYSQL_HOST}:{MYSQL_PORT}, DB={MYSQL_DB} (user={MYSQL_USER})")


## 2) Generate 200×15 Synthetic Dataset with NumPy

We create a small *sales-like* dataset with a mix of numeric, categorical, dates, and booleans.  
Columns (15 total):

1. `txn_id` — unique transaction id (int)  
2. `customer_id` — customer identifier (int)  
3. `order_date` — purchase date (date)  
4. `channel` — sales channel (`web`, `mobile`, `store`, `partner`)  
5. `region` — geographic region (`N`, `S`, `E`, `W`)  
6. `product` — product family (`A`…`E`)  
7. `unit_price` — positive float (log-normal)  
8. `quantity` — count of items (Poisson + 1)  
9. `discount` — fraction discount (0–0.5)  
10. `net_revenue` — `unit_price * quantity * (1 - discount)`  
11. `cogs` — cost of goods sold (unit_cost × quantity), where unit_cost is 50–90% of unit_price  
12. `gross_margin` — `net_revenue - cogs`  
13. `returned` — whether the order was returned (Bernoulli low p)  
14. `satisfaction` — integer survey score 1–5  
15. `coupon_applied` — whether a coupon was used (boolean)

We seed the RNG for reproducibility.


In [ ]:
# --- Generate synthetic data with NumPy ---
rng = np.random.default_rng(7)  # reproducible

n = 200
start_date = datetime(2025, 1, 1)
dates = np.array([start_date + timedelta(days=int(d)) for d in rng.integers(0, 90, size=n)])

txn_id       = np.arange(1, n+1, dtype=int)
customer_id  = rng.integers(1000, 1201, size=n, dtype=int)
channel      = rng.choice(["web", "mobile", "store", "partner"], size=n, p=[0.45,0.25,0.25,0.05])
region       = rng.choice(list("NSEW"), size=n, p=[0.3,0.25,0.3,0.15])
product      = rng.choice(["A","B","C","D","E"], size=n, p=[0.25,0.25,0.2,0.2,0.1])

# Prices ~ lognormal, clipped for reasonableness; quantities ~ Poisson + 1
unit_price   = np.clip(rng.lognormal(mean=3.0, sigma=0.4, size=n), 5, 500).round(2)
quantity     = (rng.poisson(lam=2.5, size=n) + 1).astype(int)

# Discounts uniform [0, 0.5) with more mass toward lower discounts
discount     = (rng.beta(2, 8, size=n) * 0.5).round(3)

# Derived economics
net_revenue  = (unit_price * quantity * (1 - discount)).round(2)
unit_cost    = (unit_price * rng.uniform(0.5, 0.9, size=n)).round(2)
cogs         = (unit_cost * quantity).round(2)
gross_margin = (net_revenue - cogs).round(2)

# Misc flags
returned        = rng.choice([0,1], size=n, p=[0.93, 0.07]).astype(int)
satisfaction    = rng.integers(1, 6, size=n, dtype=int)
coupon_applied  = rng.choice([0,1], size=n, p=[0.7, 0.3]).astype(int)

df = pd.DataFrame({
    "txn_id": txn_id,
    "customer_id": customer_id,
    "order_date": pd.to_datetime(dates).date,  # pure date type
    "channel": channel,
    "region": region,
    "product": product,
    "unit_price": unit_price,
    "quantity": quantity,
    "discount": discount,
    "net_revenue": net_revenue,
    "cogs": cogs,
    "gross_margin": gross_margin,
    "returned": returned,
    "satisfaction": satisfaction,
    "coupon_applied": coupon_applied
})

df.head()


## 3) Connect to MySQL, Create Database & Table

We connect to MySQL using `mysql.connector`, create the target database if needed, and set up a `sales` table with a schema matching our DataFrame.

**Schema notes:**  
- Numeric types use `INT` and `DECIMAL`.  
- Booleans stored as `TINYINT(1)`.  
- `order_date` stored as `DATE`.  
- `txn_id` as the **primary key**.


In [ ]:
# --- Connect and create database/table ---
# 1) Connect without specifying DB to create it if needed
root_cn = mysql.connect(
    host=MYSQL_HOST, port=MYSQL_PORT, user=MYSQL_USER, password=MYSQL_PASS
)
root_cn.autocommit = True
root_cur = root_cn.cursor()
root_cur.execute(f"CREATE DATABASE IF NOT EXISTS `{MYSQL_DB}`")
root_cur.close()
root_cn.close()

# 2) Connect to the target DB
cn = mysql.connect(
    host=MYSQL_HOST, port=MYSQL_PORT,
    user=MYSQL_USER, password=MYSQL_PASS,
    database=MYSQL_DB
)
cur = cn.cursor()

# 3) Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS sales (
  txn_id INT PRIMARY KEY,
  customer_id INT,
  order_date DATE,
  channel VARCHAR(16),
  region VARCHAR(8),
  product VARCHAR(16),
  unit_price DECIMAL(10,2),
  quantity INT,
  discount DECIMAL(5,4),
  net_revenue DECIMAL(12,2),
  cogs DECIMAL(12,2),
  gross_margin DECIMAL(12,2),
  returned TINYINT(1),
  satisfaction TINYINT,
  coupon_applied TINYINT(1)
)""")
cn.commit()
print("Database and table are ready.")


## 4) Bulk Insert 200 Rows into MySQL

We convert each row into a tuple and use `executemany` with a parameterized INSERT.  
This approach is safe and efficient for batch inserts (hundreds / thousands of rows).


In [ ]:
# --- Prepare records and insert ---
insert_sql = (
    """
    INSERT INTO sales (
      txn_id, customer_id, order_date, channel, region, product,
      unit_price, quantity, discount, net_revenue, cogs, gross_margin,
      returned, satisfaction, coupon_applied
    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ON DUPLICATE KEY UPDATE
      customer_id = VALUES(customer_id),
      order_date = VALUES(order_date),
      channel = VALUES(channel),
      region = VALUES(region),
      product = VALUES(product),
      unit_price = VALUES(unit_price),
      quantity = VALUES(quantity),
      discount = VALUES(discount),
      net_revenue = VALUES(net_revenue),
      cogs = VALUES(cogs),
      gross_margin = VALUES(gross_margin),
      returned = VALUES(returned),
      satisfaction = VALUES(satisfaction),
      coupon_applied = VALUES(coupon_applied)
    """
)

# Convert DataFrame rows to pure Python types (lists of tuples)
records = [
    (
        int(r.txn_id), int(r.customer_id), r.order_date, str(r.channel), str(r.region), str(r.product),
        float(r.unit_price), int(r.quantity), float(r.discount), float(r.net_revenue), float(r.cogs), float(r.gross_margin),
        int(r.returned), int(r.satisfaction), int(r.coupon_applied)
    )
    for r in df.itertuples(index=False)
]

cur.executemany(insert_sql, records)
cn.commit()
print(f"Inserted/updated {cur.rowcount} rows.")


## 5) SQL Transformations (3 Queries)

We run **three** data transformations directly in SQL and fetch results into pandas.

1. **Region × Channel metrics** — revenue, discount, satisfaction, returns, orders.  
2. **Top customers by revenue** — filter customers with ≥ 2 orders.  
3. **Daily KPIs** — revenue, COGS, margin, avg discount, return rate.


In [ ]:
# Helper: fetch cursor results into a pandas DataFrame
import pandas as pd

def fetch_df(cursor):
    cols = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()
    return pd.DataFrame(rows, columns=cols)


### 5.1 Region × Channel Metrics

In [ ]:
q1 = """
SELECT
  region,
  channel,
  ROUND(SUM(net_revenue), 2) AS revenue,
  ROUND(AVG(discount), 3)    AS avg_discount,
  ROUND(AVG(satisfaction),2) AS avg_satisfaction,
  SUM(returned)              AS returns,
  COUNT(*)                   AS orders
FROM sales
GROUP BY region, channel
ORDER BY revenue DESC;
"""
cur.execute(q1)
region_channel_df = fetch_df(cur)
region_channel_df.head()


**Explanation**  
We aggregate sales by `region` and `channel`, computing revenue, average discount & satisfaction, number of returns, and orders.  
Sorting by revenue surfaces the most valuable (region, channel) combinations.  
**Optimization options**  
- Add **indexes** on `region`, `channel` for faster grouping on larger tables.  
- Pre-aggregate into a materialized table if you run this frequently on large data.  
- Ensure numeric columns use appropriate **DECIMAL** scales to avoid rounding drift.

### 5.2 Top Customers by Revenue (≥ 2 orders)

In [ ]:
q2 = """
SELECT
  customer_id,
  COUNT(*) AS orders,
  ROUND(SUM(net_revenue), 2) AS revenue,
  ROUND(AVG(CASE WHEN net_revenue>0 THEN gross_margin/net_revenue END), 3) AS avg_margin_pct
FROM sales
GROUP BY customer_id
HAVING orders >= 2
ORDER BY revenue DESC
LIMIT 10;
"""
cur.execute(q2)
top_customers_df = fetch_df(cur)
top_customers_df


**Explanation**  
We group by `customer_id`, compute order count, revenue, and average margin% for customers with at least two orders, then rank the top 10.  
**Optimization options**  
- Index on `customer_id` speeds this query.  
- Consider **window functions** (MySQL 8+) for ranking if you need ties/dense ranks.  
- Persist VIP lists to a lookup table updated nightly for downstream apps.

### 5.3 Daily KPIs

In [ ]:
q3 = """
SELECT
  order_date AS day,
  ROUND(SUM(net_revenue), 2) AS revenue,
  ROUND(SUM(cogs), 2)        AS cogs,
  ROUND(SUM(gross_margin),2) AS margin,
  ROUND(AVG(discount), 3)    AS avg_discount,
  ROUND(SUM(returned)/COUNT(*), 3) AS return_rate
FROM sales
GROUP BY day
ORDER BY day;
"""
cur.execute(q3)
daily_kpis_df = fetch_df(cur)
daily_kpis_df.head()


**Explanation**  
We compute **daily** revenue, COGS, margin, average discount, and return rate. This is a classic KPI table for trend analysis.  
**Optimization options**  
- Add an **index on (`order_date`)**.  
- For very large tables, consider partitioning by date.  
- Use a scheduled job to write daily aggregates to a summary table.

## 6) (Optional) Load Full Table to pandas

Sometimes you want the full table for analysis in pandas. Here’s how to fetch everything safely.


In [ ]:
cur.execute("SELECT * FROM sales")
sales_df = fetch_df(cur)
sales_df.head()


## 7) Export Results to CSV

We export the transformed results to CSV files (UTF‑8). Adjust paths as needed.


In [ ]:
region_channel_df.to_csv('region_channel_metrics.csv', index=False)
top_customers_df.to_csv('top_customers.csv', index=False)
daily_kpis_df.to_csv('daily_kpis.csv', index=False)
sales_df.to_csv('sales_dump.csv', index=False)
print("Wrote: region_channel_metrics.csv, top_customers.csv, daily_kpis.csv, sales_dump.csv")


## 8) Cleanup (Close Connections)

Always close your cursor and connection when done.


In [ ]:
cur.close()
cn.close()
print("MySQL connection closed.")


## Appendix — Quick MySQL Setup (Optional)

If you need to quickly create a user and database (run in MySQL client as an admin user):

```sql
-- Create database and user (example values)
CREATE DATABASE demo_db;
CREATE USER 'demo_user'@'localhost' IDENTIFIED BY 'demo_password';
GRANT ALL PRIVILEGES ON demo_db.* TO 'demo_user'@'localhost';
FLUSH PRIVILEGES;
```

If your MySQL requires SSL or different auth plugins, consult your DBA/cloud provider docs.
